## Import libraries

In [1]:
# import relevant libraries
import pandas as pd, numpy as np, seaborn as sns
from matplotlib import pyplot as plt
import os
import datetime
import importlib

## Setup

In [2]:
os.chdir(r"C:\Data\Personal\AI ML\Markets data\Options")

In [3]:
%load_ext autoreload
%autoreload 2
%run dataparser
from dataparser import DataProcessor, DataVisualizer, SpreadStrategyCheck, DataReader, OiParticipantsVisualizer
dp = DataProcessor()
dataReader = DataReader()

## Analysis Setup

In [4]:
prevExpiry = '31/12/2020'
curExpiry = '28/01/2021'
nearExpiry = '25/02/2021'
farExpiry = '25/03/2021'
prefExpiry = curExpiry

week1, week2, week3, week4 = '07/1/2021', '14/1/2021', '21/1/2021', '28/1/2021'

In [5]:
today, yest, b4Yest = datetime.datetime(2021, 1, 8), datetime.datetime(2021, 1, 7), datetime.datetime(2021, 1, 6)
niftyBhavToday, niftyBhavYest, niftyBhavB4Yest = 14347, 14137, 14146

In [48]:
bhavToday = dataReader.readBhavcopy2(today)
bhavYest = dataReader.readBhavcopy2(yest)
bhavB4Yest = dataReader.readBhavcopy2(b4Yest)

## Options Analysis

In [54]:
opToday = dp.getOptionsForDate(today, bhavToday)
opYest = dp.getOptionsForDate(yest, bhavYest)
opB4Yest = dp.getOptionsForDate(b4Yest, bhavB4Yest)

In [55]:
opToday['EXP_DATE'].unique()

array(['14/01/2021', '21/01/2021', '28/01/2021', '04/02/2021',
       '11/02/2021', '18/02/2021', '25/02/2021', '04/03/2021',
       '10/03/2021', '25/03/2021', '24/06/2021', '30/12/2021',
       '29/12/2022', '26/06/2025'], dtype=object)

In [15]:
opTodayFilters = dp.filterOptions({
    'symbol': 'NIFTY',
    'prefExpiries': ['04/02/2021', '11/02/2021', '18/02/2021', '25/02/2021', '04/03/2021',
       '10/03/2021', '25/03/2021', '24/06/2021', '30/12/2021',
       '29/12/2022', '26/06/2025'],
    'optTypes': ['PE' ,'CE'],
    'moneyness': ['OTM'],
    'sortBy': 'Premium'
    }, opToday)
#opTodayFilters.head(5)
opTodayFilters.loc[opTodayFilters.groupby(['SYMBOL'])['Premium'].idxmax()].sort_values('Premium', ascending=False).head(1)

,SYMBOL,STR_PRICE,OPT_TYPE,CMP,CLOSE_PRICE,Premium,OPEN_INT*,LotSize,StrikePriceDiffPer,OpIntradayChange,CmpIntradayChange
2388,IDEA,14.0,CE,11.75,0.4,28000.0,38500000.0,70000,19.15,-0.1,-0.35


### Changes in OI of ITM options

In [56]:
#opComparison = dp.compareOptions(opYest, opB4Yest)
opComparison = dp.compareOptions(opToday, opYest)
opComparison.head(1)

,SYMBOL,STR_PRICE,EXP_DATE,OPT_TYPE,CLOSE_PRICE_x,CLOSE_PRICE_y,OPEN_INT*_x,Premium_x,Moneyness_x,OiChangePer,OiChangePerAbs
0,NIFTY,11200.0,14/01/2021,PE,0.65,0.5,187800.0,49.0,OTM,2991.358025,2991.358025


In [91]:
expiries = ['25/02/2021', 
            '25/03/2021', 
            '24/06/2021', 
            '30/12/2021', 
            '29/12/2022', 
            '26/06/2025']

opComparisonFiltered = dp.filterOptions({
    'returnFullset': True,
    'symbol': 'NIFTY',
    'prefExpiries': expiries,
    'optTypes': ['PE'],
    #'optTypes': ['PE', 'CE'],
    #'moneyness_x': ['ITM', 'OTM'],
    'moneyness_x': ['OTM'],
    'sortBy': 'OiChangePer',
    #'openIntThreshold': 500000
    }, opComparison)

opComparisonFiltered.head(25)

,SYMBOL,STR_PRICE,EXP_DATE,OPT_TYPE,CLOSE_PRICE_x,CLOSE_PRICE_y,OPEN_INT*_x,Premium_x,Moneyness_x,OiChangePer,OiChangePerAbs
349,NIFTY,13700.0,25/02/2021,PE,200.45,245.55,82650.0,15034.0,OTM,302.189781,302.189781
427,NIFTY,13000.0,24/06/2021,PE,293.15,317.70,8325.0,21986.0,OTM,226.470588,226.470588
334,NIFTY,12700.0,25/02/2021,PE,44.65,56.90,49950.0,3349.0,OTM,175.206612,175.206612
367,NIFTY,14300.0,25/02/2021,PE,399.70,476.50,28575.0,29978.0,OTM,168.309859,168.309859
365,NIFTY,14250.0,25/02/2021,PE,378.60,454.60,1200.0,28395.0,OTM,166.666667,166.666667
333,NIFTY,12600.0,25/02/2021,PE,38.15,48.85,61050.0,2861.0,OTM,128.011204,128.011204
335,NIFTY,12800.0,25/02/2021,PE,52.30,66.10,67875.0,3922.0,OTM,86.213992,86.213992
411,NIFTY,14200.0,25/03/2021,PE,437.30,501.00,4950.0,32798.0,OTM,60.975610,60.975610
351,NIFTY,13800.0,25/02/2021,PE,229.75,275.70,248325.0,17231.0,OTM,56.179245,56.179245
355,NIFTY,14000.0,25/02/2021,PE,289.40,348.75,504825.0,21705.0,OTM,54.985033,54.985033


In [ ]:
#opComparisonFiltered['SYMBOL'].unique()
#Symbol = 'IDEA'
#opComparison[(opComparison['SYMBOL'] == Symbol) & (opComparison['EXP_DATE'] == prefExpiry)].sort_values('OPEN_INT*_x', ascending=False).head(25)

In [ ]:
fdm = dataReader.readFundamentals('data/Stocks Research-2020-07-25.csv')

In [ ]:
opTodayFdmMerged = dp.mergeFundamentalsAndOp(opToday, fdm)

In [ ]:
expiry = prefExpiry
opTodayFdmMergedFiltered = dp.filterOptions({
        'prefExpiry': expiry, 
        'strikePriceThreshold': 10,
        'optTypes': ['CE', 'PE'],
        'moneyness': ['OTM'],
        #'openIntThreshold': 50000,
        '52weekLowThreshold': 0.3,
        'sortBy': 'Premium',
    }, opTodayFdmMerged)
opTodayFdmMergedFiltered.loc[opTodayFdmMergedFiltered.groupby(['SYMBOL'])['Premium'].idxmax()].sort_values('Premium', ascending=False).head(1)

In [ ]:
opTodayFdmMerged['EXP_DATE'].unique()

In [ ]:
#opComparison = dp.compareOptions(opYest, opB4Yest)
opComparison = dp.compareOptions(opToday, opYest)
opComparison.head(1)

In [ ]:
opComparisonFiltered = dp.filterOptions({
    #'prefExpiry': prefExpiry,
    'optTypes': ['PE' ,'CE'],
    'moneyness_x': ['ITM', 'OTM'],
    'returnFullset': True,
    'symbol': 'NIFTY',
    'sortBy': 'OiChangePer',
    'openIntThreshold': 500000
    }, opComparison)
#opTodayFilters.head(5)
opComparisonFiltered.head(25)

In [ ]:
#opComparisonFiltered['SYMBOL'].unique()
#Symbol = 'IDEA'
#opComparison[(opComparison['SYMBOL'] == Symbol) & (opComparison['EXP_DATE'] == prefExpiry)].sort_values('OPEN_INT*_x', ascending=False).head(25)

### Historical Options Analysis

In [92]:
optionsHistory = dataReader.readOptionsDataForDateRange(datetime.datetime(2020, 12, 18), 
                                                        datetime.datetime(2021, 1, 8))

In [ ]:
buyExp, sellExp = curExpiry, curExpiry
buyStrike = 24000
sellStrike = 27000 #buyStrike

dispCols = ['SYMBOL', 'EXP_DATE', 'STR_PRICE', 'OPT_TYPE', 'CLOSE_PRICE', 'LO_PRICE', 'HI_PRICE', 'ForDate']
#OPEN_PRICE 	HI_PRICE 	LO_PRICE
filterOptionsHistory = dp.filterOptions({
    'prefExpiries': [buyExp, sellExp], #prefExpiry,
    'symbol': 'BANKNIFTY',    
    'sortBy': 'ForDate',
    'optTypes': ['PE'],
    'strikePrices': [sellStrike, buyStrike],
    'returnFullset': True
    }, optionsHistory)
#filterOptionsHistory.head(1)
filterOptionsHistory[dispCols].sort_values(['ForDate'])

dates = filterOptionsHistory.sort_values(['ForDate']).ForDate.unique()
strategy = pd.DataFrame(columns=['SYMBOL', 'STR_PRICE', 'SYMBOL', 'OPT_TYPE', 'Buy', 'Sell', 'Diff', 'ForDate'])
for date in dates:
    b = filterOptionsHistory[(filterOptionsHistory.STR_PRICE == buyStrike) & (filterOptionsHistory.ForDate == date) & (filterOptionsHistory.EXP_DATE == buyExp)]
    s = filterOptionsHistory[(filterOptionsHistory.STR_PRICE == sellStrike) & (filterOptionsHistory.ForDate == date) & (filterOptionsHistory.EXP_DATE == sellExp)]
    
    if len(s.CLOSE_PRICE.values) <= 0:
        print(f'sell option not found for {sellStrike} with {sell} on {date}')
        continue
        
    diff = s.CLOSE_PRICE.values[0] - b.CLOSE_PRICE.values[0]
    strategy = strategy.append({
                    'SYMBOL': s.SYMBOL.values[0],
                    'STR_PRICE': s.STR_PRICE.values[0],
                    'SYMBOL': s.SYMBOL.values[0],
                    'OPT_TYPE': s.OPT_TYPE.values[0],
                    'Buy': b.CLOSE_PRICE.values[0],
                    'Sell': s.CLOSE_PRICE.values[0],
                    'Diff': diff,
                    'ForDate': s.ForDate.values[0],
                    }, ignore_index=True)

strategy

In [93]:
longHistory = dataReader.readOptionsDataForDateRange(datetime.datetime(2020, 11, 1), 
                                                        datetime.datetime(2021, 1, 8))

In [ ]:
symbol = 'BANKNIFTY'
strikePrice = 22000
dates = ['2020-06-25', '2020-07-30', '2020-08-27', '2020-09-24', '2020-10-29']
optType = 'PE'
dispCols = ['SYMBOL', 'EXP_DATE', 'CLOSE_PRICE', 'OPEN_INT*', 'ForDate']

filteredLongHistory = longHistory[(longHistory.SYMBOL == symbol) & (longHistory.STR_PRICE == strikePrice) & (longHistory.OPT_TYPE == optType)]

filteredLongHistory = filteredLongHistory[(filteredLongHistory.ForDate.isin(dates))]
filteredLongHistory[dispCols].head(1)
#longHistory[(longHistory.CLOSE_PRICE < 500) & (longHistory.SYMBOL == symbol) & (longHistory.STR_PRICE == strikePrice) & (longHistory.OPT_TYPE == optType)][dispCols]

## Futures Analysis

In [ ]:
futuresToday = dp.getFuturesForDate(today, bhavToday)
futuresYest = dp.getFuturesForDate(yest, bhavYest)
futuresB4Yest = dp.getFuturesForDate(b4Yest, bhavB4Yest)

In [ ]:
futuresToday[futuresToday['SYMBOL'] == 'BAJAJFINSV'].head(1)

In [ ]:
dp.filterFutures({'prefExpiry': prefExpiry, 
                  'priceDiffPerThreshold': 0.5, 
                  'fwds': ['Premium', 'Discount'], 
                  'symbol': 'IDEA',
                  'openIntThreshold': 50000, 
                  'sortBy': 'PriceDiffPerAbs'}, futuresToday).head(1)

In [ ]:
#symbol = 'IDEA'
#dp.filterOptions({'symbol': symbol, 
#                  'prefExpiry': prefExpiry, 
#                  'sortBy': 'Premium',
#                  'moneyness': ['OTM']}, opToday).head(6)

In [ ]:
futuresComparison = dp.compareFutures(futuresToday, futuresYest)
futuresComparisonYest = dp.compareFutures(futuresYest, futuresB4Yest)

In [ ]:
def getChangedFutures(futComp, limit = 10):
    ds = futComp[(futComp['Fwd_x'] != futComp['Fwd_y']) & (futComp['EXP_DATE'] == nearExpiry)].sort_values('OiChangePerAbs', ascending=False).head(limit)
    symbols = ds['SYMBOL'].values
    df = futComp[(futComp['SYMBOL'].isin(symbols)) & (futComp['EXP_DATE'].isin([nearExpiry]))]
    return df

In [ ]:
#getChangedFutures(futuresComparisonYest)
getChangedFutures(futuresComparison, 1).sort_values(['EXP_DATE', 'Fwd_y', 'OiChangePerAbs'], ascending=False)

In [ ]:
Symbol = 'BHARTIARTL'
futuresComparison[(futuresComparison['SYMBOL'] == Symbol) & (futuresComparison['EXP_DATE'].isin([curExpiry, nearExpiry]))]

### Changes in future for a symbol

In [ ]:
#Symbols = ['KOTAKBANK']
Symbols = dp.bankniftySymbols
futuresComparisonYest[(futuresComparisonYest['SYMBOL'].isin(Symbols)) & (futuresComparisonYest['EXP_DATE'].isin([curExpiry, nearExpiry]))]

In [ ]:
futuresComparisonYest[(futuresComparisonYest['EXP_DATE'].isin([curExpiry, nearExpiry]) & (futuresComparisonYest['OiChangePerAbs'] > 50))]

### Participants OI

In [ ]:
oiParticipantsVs = OiParticipantsVisualizer()
oiParticipantsVs.init()

In [ ]:
fromDate = datetime.datetime(2020, 12, 20)
toDate = datetime.datetime(2021, 1, 8)
historical_participants_oi = dataReader.readParticipantWiseOiDataForDateRange(fromDate, toDate)
historical_participants_oi = oiParticipantsVs.add_delta_datapoints(historical_participants_oi)
#data = oiCharts(historical_participants_oi, ['FII', 'Client', 'Pro', 'DII'])

In [ ]:
#clientTypes = ['FII']
clientTypes = ['FII', 'DII', 'Pro', 'Client']
data = oiParticipantsVs.oiCharts(historical_participants_oi, clientTypes)#, months = [12])

In [ ]:
data[data['ForDate'].dt.month.isin([12, 1])].tail(12).sort_values('Client Type')
#[['Future Stock Long', 'Future Stock Short', 'Future Stock Delta', 'ForDate']]

## Equity Deliveries

In [ ]:
eqDelVis = EquityDeliveryVisualizer()

In [ ]:
fromDate = datetime.datetime(2020, 12, 1)
toDate = datetime.datetime(2021, 1, 8)
historical_equity_deliveries = dataReader.readEquityDeliveryDataForDateRange(fromDate, toDate)
historical_equity_bhavcopy = dataReader.readBhavcopyForDateRange(fromDate, toDate)

In [ ]:
hist_eq_del_bhav = dp.mergeEquityBhavAndDelivery(historical_equity_bhavcopy, historical_equity_deliveries)

In [ ]:
#symbols = ['KOTAKBANK']
symbols = dp.bankniftySymbols
eqDelVis.showDeliveryChartsForSymbols(symbols, hist_eq_del_bhav)
eqDelVis.showDeliveryPercChartsForSymbols(symbols, hist_eq_del_bhav)

In [ ]:
#symbols = ['KOTAKBANK']
symbols = dp.niftySymbols
eqDelVis.showDeliveryChartsForSymbols(symbols, hist_eq_del_bhav)
eqDelVis.showDeliveryPercChartsForSymbols(symbols, hist_eq_del_bhav)

In [ ]:
symbols = ['JSWSTEEL']
dispCols = ['SYMBOL', 'ForDate', 'DeliverableQuantityPercent', 'QuantityTraded', 'DeliverableQuantity', 'CMP']
hist_eq_del_bhav_filtered = hist_eq_del_bhav[(hist_eq_del_bhav['SYMBOL'].isin(symbols)) & (hist_eq_del_bhav['RecordType'] == 'EQ')]
hist_eq_del_bhav_filtered.tail(100)[dispCols].sort_values(['SYMBOL', 'ForDate'])